In [1]:
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [3]:
import os

In [ ]:
def load_psd_features(band_name, input_dir):

    file_path = os.path.join(input_dir, f"{band_name}.pkl")

    with open(file_path, 'rb') as f:
        band_data = pickle.load(f)

    return band_data

# Example usage
data = load_psd_features("alpha","new_data/pd vs hc")
print(len(data["data"]))  # Should print 31
print(data["data"][0].shape)  # Should print (180, 32)
print(data["label"])  # Should print [1, 1, ..., 0]

In [5]:
# Extract data and labels
X = np.array(data["data"])  # Shape: (31, 180, 32)
y = np.array(data["label"])  # Shape: (31,)

In [6]:
# Flatten each subject’s 2D matrix (180x32) into a 1D vector
X_flattened = X.reshape(X.shape[0], -1)  # New shape: (31, 180*32)

In [ ]:
X_flattened[0]

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [9]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, test_size=0.2, random_state=42, stratify=y)

# Scaling
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
X_train=X_train*1e12
X_test=X_test*1e12

In [ ]:
# Train an SVM classifier
svm_model = SVC(kernel="rbf", C=1, probability=True)  # Using RBF kernel
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)


In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(svm_model, X_flattened, y, cv=3)
print("Cross-validation accuracy:", np.mean(cv_scores))


K-Fold

In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_curve, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [13]:
def svm_kfold(data, labels, k=3):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    accuracies = []
    sensitivities = []
    specificities = []
    
    fold = 1
    for train_index, test_index in skf.split(data, labels):
        print(f"Fold {fold}")
        fold += 1

        # Split the data into train and test
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        # Scaling
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Initialize and train the SVM model
        svm_model = SVC(kernel='rbf', C=1, probability=True)
        svm_model.fit(X_train_scaled, y_train)
        
        # Make predictions
        y_pred = svm_model.predict(X_test_scaled)

        # Calculate accuracy
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)
        
        # Confusion matrix to calculate sensitivity and specificity
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)

        sensitivities.append(sensitivity)
        specificities.append(specificity)

        # Output for the current fold
        print(f"Accuracy: {acc:.4f}")
        print(f"Sensitivity (Recall): {sensitivity:.4f}")
        print(f"Specificity: {specificity:.4f}")
        print()

    # Output overall metrics
    print("=== Overall Metrics ===")
    print(f"Mean Accuracy: {np.mean(accuracies):.4f}")
    print(f"Mean Sensitivity: {np.mean(sensitivities):.4f}")
    print(f"Mean Specificity: {np.mean(specificities):.4f}")

    return np.mean(accuracies), np.mean(sensitivities), np.mean(specificities)

In [ ]:
svm_kfold(X_flattened,y)